In [37]:
import requests
import urllib.parse

from beanie import init_beanie
from app.main import User, db
from beanie.odm.operators.find.comparison import GT
from beanie.odm.operators.find.logical import Not
from beanie.odm.operators.find.array import Size
from beanie.odm.operators.find.element import Exists, Type

await init_beanie(database=db, document_models=[User])

## Programmatically Login/Logout

In [43]:
data = urllib.parse.urlencode({"username": "matthias.kornexl@7lytix.com",
                               "password": "Password01!"})

headers = {"Content-Type": "application/x-www-form-urlencoded",
           "accept": "application/json"}

result = requests.post("http://localhost:8000/auth/jwt/login",
                       headers=headers,
                       data=data)
auth_cookie = result.cookies["fastapiusersauth"]
result.status_code, auth_cookie

(200, '9hvmK9wzDJx2eo74HgGE06ekQS-9n75T2CVzr5zJPlk')

In [44]:
result = requests.post("http://localhost:8000/auth/jwt/logout",
                       cookies={"fastapiusersauth": auth_cookie})
result.status_code

200

## MongoDB Operations

### Query User

In [2]:
user = await User.get("62da9a8e750b1f99bda1bcf6")

In [4]:
experiments = set()

async for user in User.find_all():
    experiments = experiments.union(participant.experiment_name
                                    for participant in user.experiments)

experiments

set()

In [6]:
groups = set()

async for user in User.find(User.is_active == True):
    groups = groups.union(member.group.group_name for member in user.groups)
    
groups

{'7LYTIX'}

In [16]:
next(member.group
          for user in await User.find_all().to_list()
          for member in user.groups)

UserGroup(group_name='7LYTIX', group_type='company')